<div style="text-align:center; font-weight:bold; font-size:35px; line-height:1.0;">EDA: Sell Your House for More </div>

<figure style="margin-top:10px"><img src="https://www.racialequityalliance.org/wp-content/uploads/2016/10/assessors_social-1.jpg" />
<figcaption style="text-align:center; font-size:8px">source: https://www.racialequityalliance.org/jurisdictions/king-county-washington/assessors_social/</figcaption>
</figure>




# Initial Imports and Data Cleaning

## How can we get more money for our house in King County?

We are selling our house and would like to know which characteristics of a home can help improve the sale price.

In order to help answer this question, we have been provided a dataset of home sales in King County which occurred during the period of September 9, 2014 through January 10, 2015.

We will attempt to model sale prices based on the other data fields and then determine which characteristics lead to an increase in sale price.

## Our Dataset

The data was provided in a csv format.  There are 21,597 sales records and 21 columns.

Column Names and descriptions:
* **id** - unique identified for a house
* **dateDate** - house was sold
* **pricePrice** -  is prediction target
* **bedroomsNumber** -  of Bedrooms/House
* **bathroomsNumber** -  of bathrooms/bedrooms
* **sqft_livingsquare** -  footage of the home
* **sqft_lotsquare** -  footage of the lot
* **floorsTotal** -  floors (levels) in house
* **waterfront** - House which has a view to a waterfront
* **view** - Has been viewed
* **condition** - How good the condition is ( Overall )
* **grade** - overall grade given to the housing unit, based on King County grading system
* **sqft_above** - square footage of house apart from basement
* **sqft_basement** - square footage of the basement
* **yr_built** - Built Year
* **yr_renovated** - Year when house was renovated
* **zipcode** - zip
* **lat** - Latitude coordinate
* **long** - Longitude coordinate
* **sqft_living15** - The square footage of interior housing living space for the nearest 15 neighbors
* **sqft_lot15** - The square footage of the land lots of the nearest 15 neighbors

## Import Libraries and Custom Functions

The following code will import our custom functions from Mod1_Functions.py.

We also import pandas, numpy, matplotlib, seaborn, statsmodels, and scikit-learn.

In [1]:
from Mod1_Functions import *


IndentationError: expected an indented block (Mod1_Functions.py, line 306)

## Import Raw Dataset

We import the csv data into a pandas dataframe by using `pandas.read_csv()`.

In [ ]:
df_raw = pd.read_csv('kc_house_data.csv')

The code below shows the count of any data columns that are missing information.

In [ ]:
missing = df_raw.shape[0] - df_raw.count()
print(missing[missing>0])

From above, we can see that `waterfront`, `view`, and `yr_renovated` are missing values.

`date` is formatted as an object type, instead of a date (datetime64).

`sqft_basement` is also formatted as an object type, when we would expect a number format.

Let's take a look at some scatter plots of each potential X variable compared to our target `'price'`.

The custom function below accepts inputs of a dataframe and the name of the column that is the target.  Additional parameters can also be passed through to adjust the formatting.

Warnings are printed for any variables that cannot immediately be graphed as numbers, due to their datatypes.

In [ ]:
scatter_y(df_raw, 'price', ncols=2, figsize=(16,45))


In [2]:
df_raw.iloc[:, 2:].hist(figsize  = [16, 10])
plt.subplots_adjust(wspace=0.5, hspace=0.6)

NameError: name 'df_raw' is not defined

## Clean data using custom function

Our `clean_dataframe()` function takes in inputs of dataframe and a dictionary of adjustments.

We first set our `data_adjustments` dictionary to contain the fields we want to change as keys and a list of adjustments as dictionary values.

The parameters in the list are: \[datatype, value to replace, value to replace with, replacement array\]

    - datatype: must be a valid data type
    - value to replace: can be a single value string, integer, or np.nan
    - value to replace with: can be a single value or can be a list with strings containg other column names in dataframe (see replacement array below)
    - replacement array: contains a list of floats or integers, which are multiplied by the associated data field in the "value to replace with" list
    
The list items should be set to None for any parameters you do not wish to use.

We found some issues with the following data fields and decided to make some adjustments.

- `'date'` : Currently the date is formatted as a string and we would like to change it into a date (datetime64) so that we can use it in our model.


- `'bedrooms'` : There was one unusually high value of 33 bedrooms for one record.  We decided to replace that value with 4, based on the properties nearest neighbors by `'price'` and `'sqft_living'`.


- `'waterfront'`: There were 2,376 missing values in this field, which we have replaced with 'missing'.  We don't know whether these records are on the waterfront or not, but it is a significant enough proportion of records (11%), so we don't want to make an assumption about whether they are on the waterfront and also don't want to exclude them from the dataset.

In [ ]:
data_adjustments = {'date': ['datetime64', None, None, None],
                    'bedrooms': [None, 33, 4, None],
                    'waterfront': [str, np.nan, 'missing', None],
                    'view': [str, np.nan, 0, None],
                    'sqft_basement': [float, '?', ['sqft_living', 'sqft_above'], [1, -1]],
                    'floors': [None, 3.5, 3, None]
                   }

In [ ]:
df_clean = clean_dataframe(df_raw, data_adjustments)

## Add features calculated from other columns

We wondered whether there was any seasonality to home sale prices, so we needed to extract the month 

We created a `'month'` column in order to calculate a `'season'` column that will be used to categorize by season of the year.

We also create a custom binned variable `'yr_renovated_cat'` which categorizes whether the house has been renovated and whether that renovation was recent.

We have also add a dummy variable that classifies whether the house's square footage above ground is in the top 50% of the data set for square footage above ground.

After some work further on with the model, we determined that there were 4 zip codes where the average sales price was much higher than in other zipcodes.  Rather than consider every zipcode as a dummy variable, we decided to make a single variable for whether or not the property is in one of the 4 high priced zipcodes.

In [ ]:
df_clean['month'] = df_clean['date'].map(lambda x: x.month)

df_clean['season'] = df_clean['month'].apply(create_season)

df_clean['has_basement'] = df_clean['sqft_basement'].apply(lambda x: 1 if x>0 else 0)

df_clean['sqft_above_tophalf'] = df_clean['sqft_above'].apply(lambda x: 1 if x>df_clean['sqft_above'].mean() else 0)

df_clean['zip_highprice'] = df_clean['zipcode'].apply(lambda x: 1 if x in [98004, 98039, 98040, 98112] else 0)

# Set number of years to consider recent renovation
n_years = 5
df_clean['yr_renovated_cat'] = df_clean['yr_renovated'].apply(
    renovated_cat, n_years=n_years)

## Set up data fields as Categorical to avoid treating as numbers

First we use our `set_to_categorical()` function to take the list of variables shown below and convert each to a category datatype in the dataframe.

In [ ]:
categorical_columns = ['floors', 'waterfront', 'view', 'condition',
                       'grade', 'yr_renovated_cat', 'season']
# not in use: 'zipcode'
set_to_categorical(df_clean, categorical_columns)

print("Categorical Variables:")
print(df_clean.dtypes[df_clean.dtypes=='category'])


We have made a custom function `create_dummyframe()` which generates a data frame of dummy variables for each of the variables listed in the second parameter from the original data frame.

In [ ]:
df_dummy = create_dummyframe(df_clean, categorical_columns)

print('df_dummy shape: {}'.format(df_dummy.shape))
print('\nPreview:')
df_dummy.head()


Let's take a look at the distribution of prices.

The prices appear to follow a log relationship.

Let's consider evulating a log-linear relationship by adding a `'log_price'` column.

In [ ]:
df_clean.price.hist(figsize  = [16, 10])

In [ ]:
df_clean['log_price'] = np.log(df_clean['price'])

And take another look at the scatter plots compared to the new target of `'log_price'` 

In [ ]:
scatter_y(df_clean, 'log_price', )

In [ ]:
df_clean.iloc[:, 2:].hist(figsize  = [16, 10])
plt.subplots_adjust(wspace=0.5, hspace=0.6)

Taking another look at the distribution of prices after taking the log reveals that we have much closer to a normal distribution of log_price.

In [ ]:
sns.distplot(df_clean.log_price);

## Analyze Correlation between our variables

We need evaluate the correlation matrix to determine whether any of our X variables are highly correlated, which would necessitate removing at least one of them to avoid multicolinearity problems.

To make things easier, we have a function that will find any pairs of variables in the matrix with an absolute value of correlation greater than the second parameter (default = 0.75).

In [ ]:
corr_pairs = findcorrpairs(df_clean, 0.5, )
corr_pairs

# Regression Model

## Initial Set-up

From our intuition about the scatter plots above, along with removing a few variables that would be correlated, we take our first shot at linear regression.

Below is our list of initial X variables to try and model upon.

Some intuitive features you would see in the listing for the home include the number of bedrooms and bathrooms and the square footage of the home and property.

In [ ]:
x_list = ["bedrooms", 
          "bathrooms", 
          "sqft_above", 
          "sqft_basement", 
          "sqft_lot", 
          "sqft_living15", 
          "sqft_lot15"]

We first run the regression using stats model's OLS function.  Note that the dataframe of X variables must include a column of constants (1).  We have decided to use `'log_price'` as our target (Y).

In [ ]:
X = df_clean.loc[:, x_list]
X = sm.add_constant(X)
Y = df_clean['log_price']

The code below fits our model and sets it equal to the variable `model_init`.  We then take a look at the results with the `.summary()` method

In [ ]:
model_init = sm.OLS(Y, X).fit()
model_init.summary()

<font size="4">Not great...<br>
    Our correlation of 0.516 isn't very good. <br><br> However, the silver lining is that the distribution of residuals appears to be approximately normally distributed based on the histogram of residuals and Q-Q plot below. </font>

In [ ]:
resid1 = model_init.resid
resid1.hist()
fig = sm.graphics.qqplot(resid1, dist=stats.norm, line='45', fit=True, alpha=0.05)
fig.show()

## Making our model a little smarter

Based on the scatterplot of possible x variables and `'log_price'` we assess that the following variables should have the log function applied in order for their relationships with `'log_price'` to be linear. 

In [ ]:
log_list = ['sqft_above', 'sqft_basement',
            'sqft_living15', 'sqft_lot', 'sqft_lot15']

# avoid value of 0 in sqft_basement
df_clean['sqft_basement'] = df_clean['sqft_basement'] + 1

for i in log_list:
    try:
        logcount
    except:
        logcount = 1
        df_clean[i] = df_clean[i].apply(np.log)   #apply(lambda x: np.log(x))



We want to convert our date field into a number of days from an arbitrary starting point so that we can treat each day as a unit.  For lack of a better date, we start from 1/1/1970.  This date does not matter because we will normalize the variable later on.

In [ ]:
df_clean['startdate'] = pd.Timestamp('19700101')

df_clean['date_num'] = (df_clean['date'] - df_clean['startdate']).dt.days


## Removing Outliers

We observed that there were some very large outliers in the fields of `'sqft_lot'` and `'sqft_lot15'`.

Our condition below removes any outliers beyond the 99.9th percentile.

The output below the code shows that 38 total outliers were removed.

In [ ]:
f1_var = 'sqft_lot'
f2_var = 'sqft_lot15'

f1_lim = df_clean[f1_var].quantile(.999)
f2_lim = df_clean[f2_var].quantile(.999)

n1 = len(df_clean[df_clean[f1_var]>=f1_lim])
n2 = len(df_clean[df_clean[f2_var]>=f2_lim])

filter1 = df_clean[f1_var]<f1_lim
filter2 = df_clean[f2_var]<f2_lim

print('filtered out {} records with {} greater than: {}'.format(n1, f1_var, f1_lim))
print('filtered out {} records with {} greater than: {}'.format(n2, f2_var, f2_lim))

df_filter = df_clean[ filter1 & filter2 ] 

print('{} total records removed'.format(len(df_filter) - len(df_clean)))

Because our variables now contain a different number of rows, we need to recreate the dummy variable dataframe.

This can be achieved by using our `create_dummyframe()` function again, but using the `df_filter` as an input instead.

In [ ]:
df_dummy_filter = create_dummyframe(df_filter, categorical_columns)

print('df_dummy shape: {}'.format(df_dummy_filter.shape))
print('\nPreview:')
df_dummy_filter.head()

## Scaling our Variables

To make sure no variables have any added or diminished effect simply due to the magnitude of the variable, we will standardize.

Some variables are standardized using the `MinMaxScaler()`, while others use the `StandardScaler()`.

We defaulted to using the `StandardScaler()` except for those variables that can be represented well by a specific domain.  

In this case, we thought that was appropriate for the `'date_num'` and `'yr_built'` data fields because they can be thought of as positions of dates within a specific range of time.

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
standard_scaler = preprocessing.StandardScaler()


Y = df_filter['log_price']


unadj_list = ['has_basement', 'zip_highprice']

  
min_max_list = ['date_num', 'yr_built']

std_scal_list = ['sqft_lot', 'sqft_above', 
                 'sqft_living15', 'sqft_lot15',]

X_unadj = pd.DataFrame(columns=unadj_list)
X_min_max = pd.DataFrame(columns=min_max_list)
X_std_scal = pd.DataFrame(columns=std_scal_list)



for col_name in min_max_list:
    X_min_max[col_name] = df_filter[col_name]
    
for col_name in std_scal_list:
    X_std_scal[col_name] = df_filter[col_name]

for col_name in unadj_list:
    X_unadj[col_name] = df_filter[col_name]   


X_min_max = pd.DataFrame(data= min_max_scaler.fit_transform(X_min_max.values), columns=X_min_max.columns)

X_std_scal = pd.DataFrame(data= standard_scaler.fit_transform(X_std_scal.values), columns=X_std_scal.columns)




Due to filtering, some of our indices became mismatched.  However, our rows still line up, so we reset the axes to concatenate all of our various X dataframes together into one.

In [ ]:
X_min_max.reset_index(inplace=True)
X_std_scal.reset_index(inplace=True)
X_unadj.reset_index(inplace=True) 
df_dummy_filter.reset_index(inplace=True)
Y = Y.reset_index().drop(['index'], axis=1)


X_possible = pd.concat([X_min_max, X_std_scal, X_unadj, df_dummy_filter], axis=1)
X_possible.head()



We need to remove several columns.  Notably, each of our categorical column groups needs at least one removed to avoid multicolinearity issues.

In [ ]:
X_model = X_possible.drop(['floors_1.0', 'waterfront_missing', 'view_0.0', 'condition_1',
                           'grade_3', 'yr_renovated_cat_Never Renovated', 
                           'season_Winter', 'index', 'waterfront_1.0', 'yr_renovated_cat_missing',
                          'grade_6', 'waterfront_0.0', 'condition_2'], axis=1)
# 'zipcode_98001',
X_model = sm.add_constant(X_model)

In [ ]:
model_smarter = sm.OLS(Y, X_model).fit()
model_smarter.summary()

In [ ]:
X_model.head()

In [ ]:
linreg = LinearRegression()

linreg.fit(X_model, Y)

print(linreg.intercept_)
print(linreg.coef_)


## K-Fold Cross Validation

In [ ]:
model_kfold = KFold(n_splits=10, shuffle=True)

MSEs = cross_val_score(linreg, X_model, Y, scoring='neg_mean_squared_error', cv=model_kfold)

mean_MSE = np.mean(MSEs)

print(mean_MSE)
print(MSEs)

## Comparing different numbers of variables using Recursive Feature Elimination

In [ ]:
cycles = 10

n = len(X_model.columns)

mse_list = []
r2_list = []
num_var = list(range(1, n+1))

    
for i in range(n):
    selector = RFE(linreg, n_features_to_select = i+1)
    selector = selector.fit(X_model, Y['log_price'])
    selected_X = X_model.columns[selector.support_]
    df_selected = X_model.loc[:, selected_X]

    model_kfold = KFold(n_splits=10, shuffle=True)
    MSEs = cross_val_score(linreg, df_selected , Y, scoring='neg_mean_squared_error', cv=model_kfold)
    mean_MSE = np.mean(MSEs)
    model = sm.OLS(Y, df_selected).fit()
    
    r2_list.append(model.rsquared_adj)
    mse_list.append(mean_MSE)


plt.figure(figsize=(10,7))
plt.plot(r2_list, label='r squared')
plt.xlabel(r"Description of $x$ coordinate (units)")
plt.ylabel(r"Description of $y$ coordinate (units)")
plt.legend()


plt.plot(mse_list, label='mse')
plt.legend()
plt.show()

## Selection of Final Model Variables

In [ ]:
selector = RFE(linreg, n_features_to_select = 34)
selector = selector.fit(X_model, Y['log_price'])
selected_X = X_model.columns[selector.support_]
df_selected = X_model.loc[:, selected_X]

model = sm.OLS(Y, df_selected).fit()
model.summary()

In [ ]:
resid1 = model.resid
resid1.hist()
# resid1[resid1 > 1.75]
# plt.scatter(Y, resid1)
fig = sm.graphics.qqplot(resid1, dist=stats.norm, line='45', fit=True)
fig.show()

In [ ]:
model_kfold = KFold(n_splits=10, shuffle=True)
MSEs = cross_val_score(linreg, df_selected , Y, scoring='neg_mean_squared_error', cv=model_kfold)
MSEs